# Aplicação para cartão de credito

Grandes bancos comerciais hoje em dia recebem inumeras aplicações para cartão de credito as quais muitas são rejeitas e muitas são aprovadas por diversos motivos como renda, historico de credito entre outros. avaliar cada aplicação manualmente seria muitos custoso para os bancos e levaria bastante tempo. Com isso, nossa tarefa é criar um modelo de machine learning para avaliar cada futuro cliente, automatizando esse tarefa assim como os rais bancos fazer hoje em dia.

<img src='201703100023430.PNG' style='width:1000px;height:500px'>

O dataser utilizado foi do repositorio da UCI e esse notebook irá percorrer os seguintes passos:

<li>Primeiro vamos carregar e vizualizar o conjunto de dados.</li>
<li>Nós vamos perceber que exeistem diversos recursos que se misturam em númericos não númericos. em diferentes escalas e tambem existem dados ausente</li>
<li>Nós faremos uma anilise exploratorio dos dados e logo em seguinda um pre_processamento para submeter esses dados a um modelo de machine learning.</li>
<li>Por fim vamos construir um modelo de machine leaning para prever se determinado cliente terá seu cartão de credito aprovado ou rejeitas pelo banco.</li>


In [1]:
# Importando pacotes necessarios

import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Visualizando o dataset
df = pd.read_csv('crx.data', header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


## 1. inspecionando o dataset

de principo os dados parecem um pouco confuso pois todos eles foram anonimizados paara proteger a privacidade dos clientes, entretando nessa tabela temos dados que correspondem a <code>Genero</code>, <code>Idade</code>, <code>Dívida</code>, <code>Estado civil</code>, <code>cliente</code>, <code>Nivel Educacional</code>, <code>Etnia</code>, <code>Anos de Trabalho</code>, <code>Padrão Anterior</code>, <code>Empregado</code>, <code>Pontuação de Credito</code>, <code>Licença Automotiva</code>, <code>Cidadão</code>, <code>Codigo Postal</code>, <code>Renda</code> e finalmente <code>Status de Aprovação</code>. o que nós da um bom inicio para analizar nossos dados.

In [3]:
df.columns = ['Gender', 'Age', 'Debt', 'Married', 'BankCustomer', 'EducationLevel', 'Ethnicity', 'YearsEmployed', 'PriorDefault', 'Employed', 'CreditScore', 'DriversLicense', 'Citizen', 'ZipCode', 'Income', 'ApprovalStatus']

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          690 non-null    object 
 1   Age             690 non-null    object 
 2   Debt            690 non-null    float64
 3   Married         690 non-null    object 
 4   BankCustomer    690 non-null    object 
 5   EducationLevel  690 non-null    object 
 6   Ethnicity       690 non-null    object 
 7   YearsEmployed   690 non-null    float64
 8   PriorDefault    690 non-null    object 
 9   Employed        690 non-null    object 
 10  CreditScore     690 non-null    int64  
 11  DriversLicense  690 non-null    object 
 12  Citizen         690 non-null    object 
 13  ZipCode         690 non-null    object 
 14  Income          690 non-null    int64  
 15  ApprovalStatus  690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


In [5]:
df.describe()

,Debt,YearsEmployed,CreditScore,Income
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.165000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
75%,7.207500,2.625000,3.00000,395.500000
max,28.000000,28.500000,67.00000,100000.000000


## 2. Limpeza nos dados

É possivel abservar que em alguns clientes há ausencia de informação e alguns dados estão com "?". com isso, é necessario imputar novos valores para esses registros ou retirar os registros que contenham "?". Veremos se há um numero significativo de registros com "?" no nosso dataset.

In [6]:
# Transformando a informação ausente em NaN
df = df.replace('?',np.NaN)
df = df.replace('+',1)
df = df.replace('-',0)

In [7]:
# contando o numero de linhas
df.isna().sum()

Gender            12
Age               12
Debt               0
Married            6
BankCustomer       6
EducationLevel     9
Ethnicity          9
YearsEmployed      0
PriorDefault       0
Employed           0
CreditScore        0
DriversLicense     0
Citizen            0
ZipCode           13
Income             0
ApprovalStatus     0
dtype: int64

In [8]:
# covertendo Age para float
df['Age'] = df['Age'].astype('float')

Como existe um pouco mais de 5% de informação ausente, optarei por apenas retiras essas observações do dataset

In [9]:
# Retirando todas as linhas que continham informação ausente.
df.dropna(inplace=True)
df.shape

(653, 16)

In [10]:
df.isna().sum()

Gender            0
Age               0
Debt              0
Married           0
BankCustomer      0
EducationLevel    0
Ethnicity         0
YearsEmployed     0
PriorDefault      0
Employed          0
CreditScore       0
DriversLicense    0
Citizen           0
ZipCode           0
Income            0
ApprovalStatus    0
dtype: int64

## 3. Pre-processamento

Agora que os valores ausentes foram devidamente tratados, nós ainda temos algumns trabalhos antes de submeter nossos dados a um modelo de machine learning.

Temos duas tarefas principais, que são:

   <li>Transformar as variaveis não-numericas e numericas.
   <li>Deixar todos os dados na mesma escala.

       

Primeiro vamos transformar todos os dados em dados numericos, nao apenas pelo processamento ser mais rapido, como tambe alguns algoritmos de machine learning esperam receber dados numericos. faremos isso com a função get_dummies.

In [11]:
# lidando com valores booleanos
df.replace('t', 1, inplace = True)
df.replace('f', 0, inplace = True)

In [12]:
# Transformar as variaveis não-numericas e numericas.
df_gender = pd.get_dummies(df.Gender).add_prefix('gender_')
df_married = pd.get_dummies(df.Married).add_prefix('Married_')
df_bankCustomer = pd.get_dummies(df.BankCustomer).add_prefix('BankCustomer_')
df_educationLevel = pd.get_dummies(df.EducationLevel).add_prefix('EducationLevel_')
df_ethnicity = pd.get_dummies(df.Ethnicity).add_prefix('Ethnicity_')
df_citizen = pd.get_dummies(df.Citizen).add_prefix('Citizen_')

In [13]:
# Excluindo variaveis não-numericas.
df.drop(columns = ['Gender','Married','BankCustomer','EducationLevel','Ethnicity','Citizen','ZipCode'], axis = 1, inplace = True)

# Adicionando varieveis dummie.
df_dummies = pd.concat([df, df_gender, df_married, df_bankCustomer, df_educationLevel, df_ethnicity, df_citizen], axis = 1)


## 3. Pre-processamento (II)

Agora que temos todos os dados em valores numericos, vamos dividi-los em dados de treino e de teste para aplicar a normalização apenas na Features do nosso dataset e não na variavel target.

In [14]:
# Dividindo variaveis em features e target
X = df_dummies.drop(['ApprovalStatus'], axis = 1)
y = df_dummies['ApprovalStatus']

In [15]:
# Aplicando a Normalização nas features
norm = MinMaxScaler()
X_norm = norm.fit_transform(X)

In [16]:
# Dividindo em dados de Teste e Treino
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.30, random_state=42)

## 4. Criação de modelos

Essencialmente, a previsão de aprovação de um cartão de credito é uma tarefa de classificação. de acordo com o dataset de UCI temos 690 observações das quais 55.5% foram rejeitadas e 44.5% foram aprovadas.

Isso nos da um bom ponte de partida ja que nosso modelo terá que prever respeitando essa estatistica. Entretanto, qual modelo deveremos usar? nos proximos passos iremos criar alguns modelos de classificação e decidar qual o melhor deles.

### 4.1 Logistic Regression

In [17]:
# criando um modelo padrão
modelo_v1 = LogisticRegression()

# Treinando o modelo
modelo_v1.fit(X_train,y_train)

# Previsões com o modelo
y_pred_v1 = modelo_v1.predict(X_test)

# Previsões de probabilidade
y_pred_proba_v1 = modelo_v1.predict_proba(X_test)[:,1]

# Roc_auc
roc_auc_v1 = roc_auc_score(y_test,y_pred_proba_v1)

# acuracia
acuracia_v1 = accuracy_score(y_test,y_pred_v1)

In [18]:
# Matriz de confusão
confusion_matrix(y_test, y_pred_v1)

array([[90, 20],
       [10, 76]], dtype=int64)

In [19]:
# Dicionario do modelo
dict_modelo_v1 = {'Nome': 'modelo_v1',
                 'Algoritmo': 'Logisct Regression',
                 'Roc_auc':roc_auc_v1,
                 'Acuraci':acuracia_v1}

### 4.2 Decision Tree

In [20]:
# definindo modelo.
dtc = DecisionTreeClassifier()

# Definição dos parametros.
param_dist_v2 = {"criterion": ['gini', 'entropy', 'log_loss'],
              "max_depth": [8, 9, 10, 11],
              "min_samples_split": [8, 10, 11, 14],
              "min_samples_leaf": [1, 2, 3, 4, 5, 6]}


# Aplicando o RandomizedSearchCV.
randomCV = RandomizedSearchCV(dtc, param_dist_v2, n_iter = 25,random_state=123)

# Treinando os modelos
randomCV.fit(X_train, y_train)

# aplicando o melhor modelo
modelo_v2 = randomCV.best_estimator_

In [21]:
# Previsões com o modelo
y_pred_v2 = modelo_v2.predict(X_test)

# Previsões de probabilidade
y_pred_proba_v2 = modelo_v2.predict_proba(X_test)[:,1]

# Roc_auc
roc_auc_v2 = roc_auc_score(y_test,y_pred_proba_v2)

# acuracia
acuracia_v2 = accuracy_score(y_test,y_pred_v2)

In [22]:
# Matriz de Confusão.
confusion_matrix(y_test, y_pred_v2)

array([[96, 14],
       [17, 69]], dtype=int64)

In [23]:
# Dicionario do modelo
dict_modelo_v2 = {'Nome': 'modelo_v2',
                 'Algoritmo': 'Decision Tree',
                 'Roc_auc':roc_auc_v2,
                 'Acuraci':acuracia_v2}

### 4.3 KNN

In [24]:
knn = KNeighborsClassifier()

# Definição dos parâmetros
param_dist_v3 = {"n_neighbors": [1, 3, 5, 7],
              "weights": ['uniform', 'distance']}

# GridSearchCV
gridcv_v3 = GridSearchCV(knn,
                         param_dist_v3, 
                         scoring = 'roc_auc', 
                         n_jobs = -1)

# treiamento do modelo
gridcv_v3.fit(X_train,y_train)

# Melhor modelo
modelo_v3 = gridcv_v3.best_estimator_
print(modelo_v3)

KNeighborsClassifier(n_neighbors=7, weights='distance')


In [25]:
# Previsões com o modelo
y_pred_v3 = modelo_v3.predict(X_test)

# Previsões de probabilidade
y_pred_proba_v3 = modelo_v3.predict_proba(X_test)[:,1]

# Roc_auc
roc_auc_v3 = roc_auc_score(y_test,y_pred_proba_v3)

# acuracia
acuracia_v3 = accuracy_score(y_test,y_pred_v3)

In [26]:
# Matriz de Confusão.
confusion_matrix(y_test, y_pred_v3)

array([[89, 21],
       [16, 70]], dtype=int64)

In [27]:
# Dicionario do modelo
dict_modelo_v3 = {'Nome': 'modelo_v3',
                 'Algoritmo': 'KNN',
                 'Roc_auc':roc_auc_v3,
                 'Acuraci':acuracia_v3}

### 4.4 Random Forest

In [28]:
clf = RandomForestClassifier(random_state = 99)

# Definição dos parâmetros
param_dist_v4 = {"max_depth": [1, 3, 7, None],
              "max_features": [8, 9, 10, 11],
              "min_samples_split": [8, 10, 12, 14],
              "min_samples_leaf": [1, 2, 3, 4, 5],
              "bootstrap": [True, False]}

# Para o classificador criado com ExtraTrees, testamos diferentes combinações de parâmetros
rsearch = RandomizedSearchCV(clf, 
                             param_distributions = param_dist_v4, 
                             n_iter = 25, 
                             return_train_score = True)  

# Aplicando o resultado ao conjunto de dados de treino e obtendo o score
rsearch.fit(X_train, y_train)

# Resultados 
rsearch.cv_results_

# Imprimindo o melhor estimador
modelo_v4 = rsearch.best_estimator_
print(modelo_v4)

RandomForestClassifier(max_features=8, min_samples_split=12, random_state=99)


In [29]:
# Previsões com o modelo
y_pred_v4 = modelo_v4.predict(X_test)

# Previsões de probabilidade
y_pred_proba_v4 = modelo_v4.predict_proba(X_test)[:,1]

# Roc_auc
roc_auc_v4 = roc_auc_score(y_test,y_pred_proba_v4)

# acuracia
acuracia_v4 = accuracy_score(y_test,y_pred_v4)

In [30]:
# Matriz de Confusão.
confusion_matrix(y_test, y_pred_v4)

array([[95, 15],
       [15, 71]], dtype=int64)

In [31]:
# Dicionario do modelo
dict_modelo_v4 = {'Nome': 'modelo_v4',
                 'Algoritmo': 'Random Forest',
                 'Roc_auc':roc_auc_v4,
                 'Acuraci':acuracia_v4}

### 4.5 SVM

In [32]:
# criação do modelo
smv_v5 = SVC(kernel = 'poly')


param_dist_v5 = {"coef0": np.array([0.5, 1]),
              "gamma": np.array([0.001, 0.01]),
              "degree": np.array([2, 3, 4])}


# Grid Search
start = time.time()
grid_search_poly = GridSearchCV(smv_v5, param_dist_v5, cv = 3)

# Treinamento
grid_search_poly.fit(X_train, y_train)
end = time.time()
print('Tempo de Treinamento do Modelo com Grid Search:', end - start)

# Acurácia em Treino
print(f"Acurácia em Treinamento: {grid_search_poly.best_score_ :.2%}")
print("")
print(f"Hiperparâmetros Ideais: {grid_search_poly.best_params_}")

modelo_v5 = grid_search_poly.best_estimator_

Tempo de Treinamento do Modelo com Grid Search: 0.19962644577026367
Acurácia em Treinamento: 88.40%

Hiperparâmetros Ideais: {'coef0': 0.5, 'degree': 2, 'gamma': 0.01}


In [33]:
# Previsões com o modelo
y_pred_v5 = modelo_v5.predict(X_test)

# acuracia
acuracia_v5 = accuracy_score(y_test,y_pred_v5)

In [34]:
# Matriz de Confusão.
confusion_matrix(y_test, y_pred_v5)

array([[88, 22],
       [ 6, 80]], dtype=int64)

In [35]:
# Dicionario do modelo
dict_modelo_v5 = {'Nome': 'modelo_v5',
                 'Algoritmo': 'SVM',
                 'Roc_auc':np.NAN,
                 'Acuraci':acuracia_v5}

### 4.6 Gradient Boosting Classifier

In [36]:
gbc = GradientBoostingClassifier()

param_dist_v6 = {
    "loss":["log_loss"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "n_estimators":[10]
    }

start = time.time()
clf_gbc = GridSearchCV(gbc, param_dist_v6, cv=10, n_jobs=-1)

clf_gbc.fit(X_train, y_train)

end = time.time()
print('Tempo de Treinamento do Modelo com Grid Search:', end - start)

Tempo de Treinamento do Modelo com Grid Search: 69.69610691070557


In [37]:
print(f"Hiperparâmetros Ideais: {clf_gbc.best_params_}")

modelo_v6 = clf_gbc.best_estimator_

Hiperparâmetros Ideais: {'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'min_samples_split': 0.31818181818181823, 'n_estimators': 10}


In [38]:
# Previsões com o modelo
y_pred_v6 = modelo_v6.predict(X_test)

# Previsões de probabilidade
y_pred_proba_v6 = modelo_v6.predict_proba(X_test)[:,1]

# Roc_auc
roc_auc_v6 = roc_auc_score(y_test,y_pred_proba_v6)

# acuracia
acuracia_v6 = accuracy_score(y_test,y_pred_v6)

In [39]:
# Matriz de Confusão.
confusion_matrix(y_test, y_pred_v6)

array([[97, 13],
       [24, 62]], dtype=int64)

In [40]:
# Dicionario do modelo
dict_modelo_v6 = {'Nome': 'modelo_v6',
                 'Algoritmo': 'Gradient boosting',
                 'Roc_auc':roc_auc_v6,
                 'Acuraci':acuracia_v6}

In [41]:
df_modelos = pd.DataFrame([dict_modelo_v1,dict_modelo_v2,dict_modelo_v3,dict_modelo_v4,dict_modelo_v5,dict_modelo_v6])
df_modelos.sort_values(by='Acuraci', ascending = False)

,Nome,Algoritmo,Roc_auc,Acuraci
4,modelo_v5,SVM,NaN,0.857143
0,modelo_v1,Logisct Regression,0.899260,0.846939
3,modelo_v4,Random Forest,0.929175,0.846939
1,modelo_v2,Decision Tree,0.866543,0.841837
2,modelo_v3,KNN,0.880708,0.811224
5,modelo_v6,Gradient boosting,0.893552,0.811224


## 5. Conclusão

Terminamos esse notebook construindo um preditor de avaliação de cartao de credito usando 6 diferentes tipo de algoritmos de machine learning e tambem usando diferentes tecnicas de pre_processamento como normalização e variaveis dummies. usamos tambem GridSearchCV que realiza a tunação dos hiperparamentos dos algoritmos. conseguimos chegar a uma acuracia de 85% o que é bem plausivel para essa base de dados usando o algoritimo SVM o qual apresentou a melhor performace. 